In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import sys
sys.path.append("../../../libs_bell")
from lib_preprocessing import job_ids_to_result, flatten_results_jobs_list, arrange_results_list_tensored3, results_list_to_hist_list

In [3]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [4]:
from libs_qrem import MooneyEtalFilter
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [5]:
max_size = 65
max_length = 64
mooney001_mitigator_list = []
for n in range(2, max_size + 1):
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 1], mit_pattern=mit_pattern)
    mooney001_mitigator_list.append(MooneyEtalFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of mooney001_mitigator_list: ", len(mooney001_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
length of mooney001_mitigator_list:  64


In [6]:
for i in range(max_length):
    t1 = time.perf_counter()
    _ = mooney001_mitigator_list[i].apply(raw_hist_list[i + 1], threshold = 0.01)
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 8.930399999940164e-05 s )
2 th finished ( 3 qubits, 7.83359999996236e-05 s )
3 th finished ( 4 qubits, 8.614300000076014e-05 s )
4 th finished ( 5 qubits, 0.00015905000000060454 s )
5 th finished ( 6 qubits, 0.00023510699999995666 s )
6 th finished ( 7 qubits, 0.00022843700000052536 s )
7 th finished ( 8 qubits, 0.0003038429999993042 s )
8 th finished ( 9 qubits, 0.00038469599999935156 s )
9 th finished ( 10 qubits, 0.0004713450000011221 s )
10 th finished ( 11 qubits, 0.0006876540000000375 s )
11 th finished ( 12 qubits, 0.0010533920000010966 s )
12 th finished ( 13 qubits, 0.0010701429999997458 s )
13 th finished ( 14 qubits, 0.0011005330000006808 s )
14 th finished ( 15 qubits, 0.0014189259999994874 s )
15 th finished ( 16 qubits, 0.0016820489999993526 s )
16 th finished ( 17 qubits, 0.0024140970000008366 s )
17 th finished ( 18 qubits, 0.002503306999999566 s )
18 th finished ( 19 qubits, 0.0028410779999994418 s )
19 th finished ( 20 qubits, 0.0031576909999

In [7]:
mooney001_mitigator_info = []
for i in range(max_length):
    t1 = time.perf_counter()
    mooney001_mitigator_info.append({# "exact_one_norm_of_reduced_inv_A": mooney001_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                                 "mitigated_hist": mooney001_mitigator_list[i].mitigated_hist(),
                                 "x_s": mooney001_mitigator_list[i].x_s(),
                                 # "x_hat": mooney001_mitigator_list[i].x_hat(),
                                 "x_tilde": mooney001_mitigator_list[i].x_tilde(),
                                 "sum_of_x": mooney001_mitigator_list[i].sum_of_x(),
                                 # "sum_of_x_hat": mooney001_mitigator_list[i].sum_of_x_hat(),
                                 "sum_of_x_tilde": mooney001_mitigator_list[i].sum_of_x_tilde(),
                                 "indices_to_keys_vector": mooney001_mitigator_list[i].indices_to_keys_vector(),
                                 "times": mooney001_mitigator_list[i].times(),
                                 "expval": mooney001_mitigator_list[i].expval(),
                                 # "mitigation_stddev": mooney001_mitigator_list[i].mitigation_stddev(norm_type = "exact"),
                                 })
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 2.132200000026785e-05 s )
2 th finished ( 3 qubits, 1.0650000000111959e-05 s )
3 th finished ( 4 qubits, 6.454999999405686e-06 s )
4 th finished ( 5 qubits, 7.888000000733086e-06 s )
5 th finished ( 6 qubits, 7.1300000001883745e-06 s )
6 th finished ( 7 qubits, 6.752999999193321e-06 s )
7 th finished ( 8 qubits, 5.754000000024462e-06 s )
8 th finished ( 9 qubits, 5.715000000350301e-06 s )
9 th finished ( 10 qubits, 5.911000000580202e-06 s )
10 th finished ( 11 qubits, 6.164000000197234e-06 s )
11 th finished ( 12 qubits, 7.408000000097559e-06 s )
12 th finished ( 13 qubits, 6.199999999623174e-06 s )
13 th finished ( 14 qubits, 7.2860000006613745e-06 s )
14 th finished ( 15 qubits, 6.654999999966549e-06 s )
15 th finished ( 16 qubits, 1.32339999989739e-05 s )
16 th finished ( 17 qubits, 9.28500000085819e-06 s )
17 th finished ( 18 qubits, 8.11999999861257e-06 s )
18 th finished ( 19 qubits, 7.321000000004574e-06 s )
19 th finished ( 20 qubits, 7.289999999215979

In [8]:
with open("./pkls/mooney001_mitigator_info.pkl", "wb") as f:
    pickle.dump(mooney001_mitigator_info, f)